In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import xgboost as xgb
import catboost as cbt

pd.set_option('display.float_format', lambda x: '%.2f' % x)


DATA_PATH = '/Users/kaiqu/kaggle-datasets/train.csv'

# Load data from CSV
df = pd.read_csv(DATA_PATH)

In [2]:
xgb.__version__

'2.0.2'

In [3]:
cbt.__version__

'1.2.2'

In [4]:
df.info # seems to be useless

<bound method DataFrame.info of          stock_id  date_id  seconds_in_bucket  imbalance_size  \
0               0        0                  0      3180602.69   
1               1        0                  0       166603.91   
2               2        0                  0       302879.87   
3               3        0                  0     11917682.27   
4               4        0                  0       447549.96   
...           ...      ...                ...             ...   
5237975       195      480                540      2440722.89   
5237976       196      480                540       349510.47   
5237977       197      480                540            0.00   
5237978       198      480                540      1000898.84   
5237979       199      480                540      1884285.71   

         imbalance_buy_sell_flag  reference_price  matched_size  far_price  \
0                              1             1.00   13380276.64        NaN   
1                             -

In [5]:
df.describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,5237980.00,5237980.00,5237980.00,5237760.00,5237980.00,5237760.00,5237760.00,2343638.00,2380800.00,5237760.00,5237980.00,5237760.00,5237980.00,5237760.00,5237892.00,5237980.00
mean,99.29,241.51,270.00,5715293.10,-0.01,1.00,45100245.28,1.00,1.00,1.00,51813.59,1.00,53575.68,1.00,-0.05,13310.05
std,57.87,138.53,158.75,20515906.30,0.89,0.00,139841291.28,0.72,0.01,0.00,111421.41,0.00,129355.44,0.00,9.45,7619.27
min,0.00,0.00,0.00,0.00,-1.00,0.94,4316.61,0.00,0.79,0.93,0.00,0.94,0.00,0.94,-385.29,0.00
25%,49.00,122.00,130.00,84534.15,-1.00,1.00,5279575.17,1.00,1.00,1.00,7374.72,1.00,7823.70,1.00,-4.56,6729.00
50%,99.00,242.00,270.00,1113604.44,0.00,1.00,12882638.56,1.00,1.00,1.00,21969.00,1.00,23017.92,1.00,-0.06,13345.00
75%,149.00,361.00,410.00,4190951.00,1.00,1.00,32700130.58,1.00,1.00,1.00,55831.68,1.00,57878.41,1.00,4.41,19907.00
max,199.00,480.00,540.00,2982027811.68,1.00,1.08,7713682054.05,437.95,1.31,1.08,30287840.00,1.08,54405000.00,1.08,446.07,26454.00


In [6]:
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,1.00,13380276.64,NaN,NaN,1.00,60651.50,1.00,8493.03,1.00,-3.03,0,0_0_0
1,1,0,0,166603.91,-1,1.00,1642214.25,NaN,NaN,1.00,3233.04,1.00,20605.09,1.00,-5.52,0,0_0_1
2,2,0,0,302879.87,-1,1.00,1819368.03,NaN,NaN,1.00,37956.00,1.00,18995.00,1.00,-8.39,0,0_0_2
3,3,0,0,11917682.27,-1,1.00,18389745.62,NaN,NaN,1.00,2324.90,1.00,479032.40,1.00,-4.01,0,0_0_3
4,4,0,0,447549.96,-1,1.00,17860614.95,NaN,NaN,1.00,16485.54,1.00,434.10,1.00,-7.35,0,0_0_4


In [7]:
df.count()

stock_id                   5237980
date_id                    5237980
seconds_in_bucket          5237980
imbalance_size             5237760
imbalance_buy_sell_flag    5237980
reference_price            5237760
matched_size               5237760
far_price                  2343638
near_price                 2380800
bid_price                  5237760
bid_size                   5237980
ask_price                  5237760
ask_size                   5237980
wap                        5237760
target                     5237892
time_id                    5237980
row_id                     5237980
dtype: int64

In [8]:
df[df['far_price'] != 'NaN'].count()

stock_id                   5237980
date_id                    5237980
seconds_in_bucket          5237980
imbalance_size             5237760
imbalance_buy_sell_flag    5237980
reference_price            5237760
matched_size               5237760
far_price                  2343638
near_price                 2380800
bid_price                  5237760
bid_size                   5237980
ask_price                  5237760
ask_size                   5237980
wap                        5237760
target                     5237892
time_id                    5237980
row_id                     5237980
dtype: int64

In [9]:
df.dtypes

stock_id                     int64
date_id                      int64
seconds_in_bucket            int64
imbalance_size             float64
imbalance_buy_sell_flag      int64
reference_price            float64
matched_size               float64
far_price                  float64
near_price                 float64
bid_price                  float64
bid_size                   float64
ask_price                  float64
ask_size                   float64
wap                        float64
target                     float64
time_id                      int64
row_id                      object
dtype: object

In [10]:
df.select_dtypes(include='object') # ? what is the meaning of the row_id?

,row_id
0,0_0_0
1,0_0_1
2,0_0_2
3,0_0_3
4,0_0_4
...,...
5237975,480_540_195
5237976,480_540_196
5237977,480_540_197
5237978,480_540_198


In [11]:
df.isnull().sum() # ? why there is a 220 null data? and why there is 2894342 and 2857180 null data?

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
target                          88
time_id                          0
row_id                           0
dtype: int64

In [12]:
# query the df rows where the imbalance_size column is null
df[df['imbalance_size'].isnull()]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
369508,131,35,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1925,35_0_131
369700,131,35,10,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1926,35_10_131
369892,131,35,20,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1927,35_20_131
370084,131,35,30,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1928,35_30_131
370276,131,35,40,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1929,35_40_131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774999,19,438,500,NaN,-1,NaN,NaN,NaN,NaN,NaN,115491.18,NaN,1990.10,NaN,3.32,24140,438_500_19
4775199,19,438,510,NaN,-1,NaN,NaN,NaN,NaN,NaN,189040.50,NaN,26283.84,NaN,-5.05,24141,438_510_19
4775399,19,438,520,NaN,-1,NaN,NaN,NaN,NaN,NaN,1392.93,NaN,43799.80,NaN,-0.11,24142,438_520_19
4775599,19,438,530,NaN,-1,NaN,NaN,NaN,NaN,NaN,13531.32,NaN,26881.20,NaN,-1.69,24143,438_530_19


In [13]:
# get the set of stock ids where the imbalance_size column is null
df[df['target'].isnull()].stock_id.value_counts()

stock_id
158    55
131    31
101     1
19      1
Name: count, dtype: int64

In [14]:
# investigate where the target is null
df.query('(target != target) & (stock_id == 158)')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
4215138,158,388,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21340,388_0_158
4215338,158,388,10,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21341,388_10_158
4215538,158,388,20,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21342,388_20_158
4215738,158,388,30,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21343,388_30_158
4215938,158,388,40,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21344,388_40_158
4216138,158,388,50,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21345,388_50_158
4216338,158,388,60,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21346,388_60_158
4216538,158,388,70,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21347,388_70_158
4216738,158,388,80,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21348,388_80_158
4216938,158,388,90,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21349,388_90_158


In [15]:
df.query('(target != target) & (stock_id == 131)')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
369508,131,35,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1925,35_0_131
369700,131,35,10,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1926,35_10_131
369892,131,35,20,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1927,35_20_131
370084,131,35,30,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1928,35_30_131
370276,131,35,40,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1929,35_40_131
370468,131,35,50,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1930,35_50_131
370660,131,35,60,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1931,35_60_131
370852,131,35,70,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1932,35_70_131
371044,131,35,80,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1933,35_80_131
371236,131,35,90,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1934,35_90_131


In [16]:
df.query('(target != target) & (stock_id == 101)')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
3555081,101,328,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,18040,328_0_101


In [17]:
df.query('(target != target) & (stock_id == 19)')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
4764999,19,438,0,NaN,-1,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,24090,438_0_19


In [18]:
# investigate where the imbalance_size is null
df.query('(imbalance_size != imbalance_size) & (stock_id == 131)')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
369508,131,35,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1925,35_0_131
369700,131,35,10,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1926,35_10_131
369892,131,35,20,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1927,35_20_131
370084,131,35,30,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1928,35_30_131
370276,131,35,40,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1929,35_40_131
370468,131,35,50,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1930,35_50_131
370660,131,35,60,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1931,35_60_131
370852,131,35,70,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1932,35_70_131
371044,131,35,80,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1933,35_80_131
371236,131,35,90,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1934,35_90_131


In [19]:
# 101 seems to be the one where target is not Nah whereas imbalance size is Nah
df.query('(imbalance_size != imbalance_size) & (stock_id == 101)')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
3555081,101,328,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,18040,328_0_101
3555281,101,328,10,NaN,0,NaN,NaN,NaN,NaN,NaN,181391.80,NaN,790.04,NaN,6.43,18041,328_10_101
3555481,101,328,20,NaN,0,NaN,NaN,NaN,NaN,NaN,181391.80,NaN,790.04,NaN,6.60,18042,328_20_101
3555681,101,328,30,NaN,0,NaN,NaN,NaN,NaN,NaN,23660.40,NaN,790.04,NaN,6.35,18043,328_30_101
3555881,101,328,40,NaN,0,NaN,NaN,NaN,NaN,NaN,23660.40,NaN,790.04,NaN,3.12,18044,328_40_101
3556081,101,328,50,NaN,0,NaN,NaN,NaN,NaN,NaN,23660.40,NaN,790.04,NaN,3.43,18045,328_50_101
3556281,101,328,60,NaN,0,NaN,NaN,NaN,NaN,NaN,78869.00,NaN,790.04,NaN,10.09,18046,328_60_101
3556481,101,328,70,NaN,0,NaN,NaN,NaN,NaN,NaN,78898.00,NaN,3952.15,NaN,9.21,18047,328_70_101
3556681,101,328,80,NaN,0,NaN,NaN,NaN,NaN,NaN,78899.00,NaN,3952.15,NaN,-4.26,18048,328_80_101
3556881,101,328,90,NaN,0,NaN,NaN,NaN,NaN,NaN,103362.93,NaN,3952.15,NaN,-17.29,18049,328_90_101


In [20]:
df.query('(imbalance_size != imbalance_size) & (stock_id == 158)')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
4215138,158,388,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21340,388_0_158
4215338,158,388,10,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21341,388_10_158
4215538,158,388,20,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21342,388_20_158
4215738,158,388,30,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21343,388_30_158
4215938,158,388,40,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21344,388_40_158
4216138,158,388,50,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21345,388_50_158
4216338,158,388,60,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21346,388_60_158
4216538,158,388,70,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21347,388_70_158
4216738,158,388,80,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21348,388_80_158
4216938,158,388,90,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,21349,388_90_158


In [21]:
df.query('(imbalance_size != imbalance_size) & (stock_id == 19)')

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
4764999,19,438,0,NaN,-1,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,24090,438_0_19
4765199,19,438,10,NaN,-1,NaN,NaN,NaN,NaN,NaN,993.40,NaN,3776.44,NaN,3.80,24091,438_10_19
4765399,19,438,20,NaN,-1,NaN,NaN,NaN,NaN,NaN,993.40,NaN,3776.44,NaN,0.22,24092,438_20_19
4765599,19,438,30,NaN,-1,NaN,NaN,NaN,NaN,NaN,15097.40,NaN,3974.80,NaN,-1.30,24093,438_30_19
4765799,19,438,40,NaN,-1,NaN,NaN,NaN,NaN,NaN,15494.70,NaN,4173.54,NaN,-1.90,24094,438_40_19
4765999,19,438,50,NaN,-1,NaN,NaN,NaN,NaN,NaN,12714.24,NaN,3776.44,NaN,-5.86,24095,438_50_19
4766199,19,438,60,NaN,-1,NaN,NaN,NaN,NaN,NaN,13111.56,NaN,3776.44,NaN,-6.20,24096,438_60_19
4766399,19,438,70,NaN,-1,NaN,NaN,NaN,NaN,NaN,30199.36,NaN,3777.39,NaN,-11.91,24097,438_70_19
4766599,19,438,80,NaN,-1,NaN,NaN,NaN,NaN,NaN,4965.25,NaN,4572.63,NaN,-9.21,24098,438_80_19
4766799,19,438,90,NaN,-1,NaN,NaN,NaN,NaN,NaN,4965.25,NaN,4572.63,NaN,-8.68,24099,438_90_19


In [22]:
# get df rows where the roll is null
df[df['imbalance_size'].isna()]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
369508,131,35,0,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1925,35_0_131
369700,131,35,10,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1926,35_10_131
369892,131,35,20,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1927,35_20_131
370084,131,35,30,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1928,35_30_131
370276,131,35,40,NaN,0,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.00,NaN,NaN,1929,35_40_131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4774999,19,438,500,NaN,-1,NaN,NaN,NaN,NaN,NaN,115491.18,NaN,1990.10,NaN,3.32,24140,438_500_19
4775199,19,438,510,NaN,-1,NaN,NaN,NaN,NaN,NaN,189040.50,NaN,26283.84,NaN,-5.05,24141,438_510_19
4775399,19,438,520,NaN,-1,NaN,NaN,NaN,NaN,NaN,1392.93,NaN,43799.80,NaN,-0.11,24142,438_520_19
4775599,19,438,530,NaN,-1,NaN,NaN,NaN,NaN,NaN,13531.32,NaN,26881.20,NaN,-1.69,24143,438_530_19


In [23]:
df['imbalance_size'].isna()

0          False
1          False
2          False
3          False
4          False
           ...  
5237975    False
5237976    False
5237977    False
5237978    False
5237979    False
Name: imbalance_size, Length: 5237980, dtype: bool

In [24]:
df[(df['imbalance_size'].isna()) & (df['imbalance_buy_sell_flag'] == 1)]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
3557881,101,328,140,NaN,1,NaN,NaN,NaN,NaN,NaN,2367.21,NaN,82971.00,NaN,-7.27,18054,328_140_101
3558081,101,328,150,NaN,1,NaN,NaN,NaN,NaN,NaN,788.26,NaN,78951.00,NaN,11.08,18055,328_150_101
3558281,101,328,160,NaN,1,NaN,NaN,NaN,NaN,NaN,788.26,NaN,78920.00,NaN,15.03,18056,328_160_101
3558481,101,328,170,NaN,1,NaN,NaN,NaN,NaN,NaN,788.26,NaN,789.20,NaN,9.59,18057,328_170_101
3558681,101,328,180,NaN,1,NaN,NaN,NaN,NaN,NaN,157654.00,NaN,17362.18,NaN,5.70,18058,328_180_101
3558881,101,328,190,NaN,1,NaN,NaN,NaN,NaN,NaN,157654.00,NaN,79010.00,NaN,-1.92,18059,328_190_101
3559081,101,328,200,NaN,1,NaN,NaN,NaN,NaN,NaN,5517.89,NaN,78953.00,NaN,8.09,18060,328_200_101
3559281,101,328,210,NaN,1,NaN,NaN,NaN,NaN,NaN,39415.00,NaN,78950.00,NaN,-1.58,18061,328_210_101
3559481,101,328,220,NaN,1,NaN,NaN,NaN,NaN,NaN,47298.00,NaN,82108.00,NaN,-4.00,18062,328_220_101
3559681,101,328,230,NaN,1,NaN,NaN,NaN,NaN,NaN,55181.00,NaN,78936.00,NaN,-3.05,18063,328_230_101


In [25]:
df['imbalance_buy_sell_flag'] == -1

0          False
1           True
2           True
3           True
4           True
           ...  
5237975     True
5237976     True
5237977    False
5237978    False
5237979     True
Name: imbalance_buy_sell_flag, Length: 5237980, dtype: bool

In [26]:
df.iloc[5237979]

stock_id                           199
date_id                            480
seconds_in_bucket                  540
imbalance_size              1884285.71
imbalance_buy_sell_flag             -1
reference_price                   1.00
matched_size               24073677.32
far_price                         1.00
near_price                        1.00
bid_price                         1.00
bid_size                     250081.44
ask_price                         1.00
ask_size                     300167.56
wap                               1.00
target                           -6.53
time_id                          26454
row_id                     480_540_199
Name: 5237979, dtype: object

In [27]:
df[df['stock_id'] == 1]

# 26455 / 600 = 4.29

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
1,1,0,0,166603.91,-1,1.00,1642214.25,NaN,NaN,1.00,3233.04,1.00,20605.09,1.00,-5.52,0,0_0_1
192,1,0,10,165923.20,-1,1.00,1642894.96,NaN,NaN,1.00,3743.52,1.00,33717.42,1.00,-1.62,1,0_10_1
383,1,0,20,165923.20,-1,1.00,1642894.96,NaN,NaN,1.00,3743.52,1.00,33717.42,1.00,-6.46,2,0_20_1
574,1,0,30,165923.20,-1,1.00,1642894.96,NaN,NaN,1.00,4254.00,1.00,3574.83,1.00,-5.15,3,0_30_1
765,1,0,40,162349.47,-1,1.00,1646468.69,NaN,NaN,1.00,21270.00,1.00,3574.83,1.00,-6.75,4,0_40_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5236981,1,480,500,88468.41,-1,1.00,7635810.59,1.00,1.00,1.00,176306.44,1.00,64345.00,1.00,2.85,26450,480_500_1
5237181,1,480,510,310778.50,-1,1.00,7413500.50,1.00,1.00,1.00,1324.33,1.00,47135.70,1.00,3.17,26451,480_510_1
5237381,1,480,520,310778.50,-1,1.00,7413500.50,1.00,1.00,1.00,3783.80,1.00,85374.30,1.00,5.69,26452,480_520_1
5237581,1,480,530,310778.50,-1,1.00,7413500.50,1.00,1.00,1.00,7567.60,1.00,112065.60,1.00,10.65,26453,480_530_1


In [28]:
df[df['stock_id'] == 2]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
2,2,0,0,302879.87,-1,1.00,1819368.03,NaN,NaN,1.00,37956.00,1.00,18995.00,1.00,-8.39,0,0_0_2
193,2,0,10,184766.21,-1,1.00,1937481.68,NaN,NaN,1.00,37956.00,1.00,379.90,1.00,-7.46,1,0_10_2
384,2,0,20,169764.64,-1,1.00,1952483.25,NaN,NaN,1.00,18974.00,1.00,569.61,1.00,-6.38,2,0_20_2
575,2,0,30,169764.64,-1,1.00,1952483.25,NaN,NaN,1.00,18974.00,1.00,189.87,1.00,-5.82,3,0_30_2
766,2,0,40,169764.64,-1,1.00,1952483.25,NaN,NaN,1.00,18974.00,1.00,49368.80,1.00,-0.28,4,0_40_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5236982,2,480,500,376011.29,-1,1.00,10889581.97,1.00,1.00,1.00,23248.80,1.00,190117.80,1.00,-4.45,26450,480_500_2
5237182,2,480,510,376011.29,-1,1.00,10889581.97,1.00,1.00,1.00,71683.80,1.00,214149.00,1.00,-5.56,26451,480_510_2
5237382,2,480,520,260569.23,-1,1.00,11005024.03,1.00,1.00,1.00,968.55,1.00,105034.18,1.00,-5.31,26452,480_520_2
5237582,2,480,530,184125.24,-1,1.00,11081468.02,1.00,1.00,1.00,581.04,1.00,63740.46,1.00,-5.24,26453,480_530_2


In [29]:
# investigate a normal number stock
df[df['stock_id'] == 100] # 26455 / 55 = 481

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
97,100,0,0,0.00,0,1.00,80691.87,NaN,NaN,1.00,4807.78,1.00,6241.62,1.00,-5.23,0,0_0_100
288,100,0,10,0.00,0,1.00,80691.87,NaN,NaN,1.00,1566.58,1.00,2730.03,1.00,-3.02,1,0_10_100
479,100,0,20,0.00,0,1.00,80691.87,NaN,NaN,1.00,1566.58,1.00,2730.03,1.00,3.05,2,0_20_100
670,100,0,30,0.00,0,1.00,80691.87,NaN,NaN,1.00,16422.08,1.00,5433.03,1.00,6.65,3,0_30_100
861,100,0,40,0.00,0,1.00,80691.87,NaN,NaN,1.00,1702.26,1.00,5433.03,1.00,26.32,4,0_40_100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237080,100,480,500,0.00,0,1.00,1290339.72,1.00,1.00,1.00,29699.68,1.00,63961.20,1.00,-6.74,26450,480_500_100
5237280,100,480,510,0.00,0,1.00,1290339.72,1.00,1.00,1.00,30085.20,1.00,63961.20,1.00,-11.86,26451,480_510_100
5237480,100,480,520,0.00,0,1.00,1290339.72,1.00,1.00,1.00,37263.68,1.00,63961.20,1.00,-13.17,26452,480_520_100
5237680,100,480,530,0.00,0,1.00,1290339.72,1.00,1.00,1.00,42631.68,1.00,51736.20,1.00,-15.16,26453,480_530_100


In [30]:
DATE_RANGE = 480
EXPECTED_DAYS = 55
for i in range(DATE_RANGE + 1):
    if len(df[(df['date_id'] == i) & (df['stock_id'] == 100)]) != EXPECTED_DAYS:       
        print(i, len(df[df['date_id'] == i]))
    if i == 480:
        print(len(df[(df['date_id'] == i) & (df['stock_id'] == 100)]))

55


In [31]:
df[(df['stock_id'] == 100) & (df['date_id'] == 1)].to_csv('normal_stock_day_1.csv')

In [32]:
df[df['stock_id'] == 100].max()

stock_id                           100
date_id                            480
seconds_in_bucket                  540
imbalance_size            235051642.59
imbalance_buy_sell_flag              1
reference_price                   1.03
matched_size              428930860.41
far_price                         1.21
near_price                        1.13
bid_price                         1.03
bid_size                     617043.20
ask_price                         1.03
ask_size                    3950411.40
wap                               1.03
target                          234.57
time_id                          26454
row_id                        9_90_100
dtype: object

In [33]:
df.max()

stock_id                            199
date_id                             480
seconds_in_bucket                   540
imbalance_size            2982027811.68
imbalance_buy_sell_flag               1
reference_price                    1.08
matched_size              7713682054.05
far_price                        437.95
near_price                         1.31
bid_price                          1.08
bid_size                    30287840.00
ask_price                          1.08
ask_size                    54405000.00
wap                                1.08
target                           446.07
time_id                           26454
row_id                          9_90_99
dtype: object

In [34]:
# investigate the missing data for stock 199
df[df['stock_id'] == 199] # 21615 / 55 = 406.4

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
937451,199,88,0,20887917.73,1,1.00,5487274.58,NaN,NaN,1.00,14108.05,1.00,5002.00,1.00,-20.32,4840,88_0_199
937648,199,88,10,20837918.65,1,1.00,5537273.66,NaN,NaN,1.00,31162.56,1.00,44964.00,1.00,-10.24,4841,88_10_199
937845,199,88,20,20915667.22,1,1.00,5537273.66,NaN,NaN,1.00,47405.00,1.00,91939.54,1.00,-9.18,4842,88_20_199
938042,199,88,30,20915667.22,1,1.00,5537273.66,NaN,NaN,1.00,73672.76,1.00,32534.80,1.00,-6.46,4843,88_30_199
938239,199,88,40,20915667.22,1,1.00,5537273.66,NaN,NaN,1.00,16784.62,1.00,40019.80,1.00,-6.09,4844,88_40_199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237179,199,480,500,2661783.46,-1,1.00,23969215.23,1.00,1.00,1.00,122246.18,1.00,425296.17,1.00,-7.21,26450,480_500_199
5237379,199,480,510,2658917.61,-1,1.00,23972081.09,1.00,1.00,1.00,677012.04,1.00,347268.87,1.00,-9.75,26451,480_510_199
5237579,199,480,520,1352558.88,-1,1.00,23978379.68,1.00,1.00,1.00,225361.65,1.00,194630.52,1.00,3.63,26452,480_520_199
5237779,199,480,530,1352558.88,-1,1.00,23978379.68,1.00,1.00,1.00,285559.05,1.00,214513.32,1.00,4.76,26453,480_530_199


In [35]:
print(len(df[df['stock_id'] == 199]))

21615


In [36]:
MAX_STOCKS = 199
EXPECTED_ROWS = 26455 # 26455 / 55 = 51.64
for stock_id in range(1, MAX_STOCKS + 1): 
    if len(df[df['stock_id'] == stock_id]) != EXPECTED_ROWS:
        print(stock_id, len(df[df['stock_id'] == stock_id]))

#! several missing data for some stocks - 69, 73, 78, 79, 99, 102, 135, 150, 153, 156, 199 

69 24420
73 26400
78 26235
79 16500
99 26400
102 10230
135 15950
150 23210
153 22605
156 24420
199 21615


In [37]:
# investigate the missing data overall
df[df['stock_id'] == 69].describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,10514.00,11100.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00
mean,69.00,258.50,270.00,1190557.34,0.17,1.00,4318962.44,1.00,1.00,1.00,225061.10,1.00,229451.30,1.00,-0.08,14244.50
std,0.00,128.17,158.75,4231205.96,0.86,0.00,14973285.30,0.04,0.01,0.00,196054.72,0.00,201613.21,0.00,12.51,7049.59
min,69.00,37.00,0.00,0.00,-1.00,0.97,255469.58,0.44,0.90,0.97,45.78,0.97,17.68,0.97,-107.65,2035.00
25%,69.00,147.75,130.00,29547.65,-1.00,1.00,1190966.31,1.00,1.00,1.00,100271.51,1.00,96921.61,1.00,-6.17,8139.75
50%,69.00,258.50,270.00,401185.54,0.00,1.00,2142625.87,1.00,1.00,1.00,180360.73,1.00,178837.55,1.00,-0.22,14244.50
75%,69.00,369.25,410.00,1088944.72,1.00,1.00,3596301.60,1.01,1.00,1.00,295145.85,1.00,305630.33,1.00,5.80,20349.25
max,69.00,480.00,540.00,117946398.66,1.00,1.02,275997753.91,2.09,1.10,1.02,4221028.35,1.02,2163217.21,1.02,203.05,26454.00


In [38]:
df[df['stock_id'] == 199].describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,21615.00,21615.00,21615.00,21615.00,21615.00,21615.00,21615.00,9757.00,9825.00,21615.00,21615.00,21615.00,21615.00,21615.00,21615.00,21615.00
mean,199.00,284.00,270.00,3158094.28,-0.02,1.00,24357417.42,1.00,1.00,1.00,81383.17,1.00,83447.15,1.00,0.13,15647.00
std,0.00,113.45,158.75,10780617.62,0.89,0.00,43516515.88,0.02,0.01,0.00,107257.48,0.00,86952.58,0.00,9.06,6239.86
min,199.00,88.00,0.00,0.00,-1.00,0.99,2835012.52,0.75,0.90,0.99,27.41,0.99,25.59,0.99,-60.83,4840.00
25%,199.00,186.00,130.00,183602.44,-1.00,1.00,9888021.03,1.00,1.00,1.00,32929.76,1.00,34059.98,1.00,-5.01,10243.50
50%,199.00,284.00,270.00,1317370.40,0.00,1.00,15805715.56,1.00,1.00,1.00,56211.68,1.00,57915.42,1.00,0.09,15647.00
75%,199.00,382.00,410.00,3336578.50,1.00,1.00,23463464.63,1.00,1.00,1.00,100812.62,1.00,102294.44,1.00,5.10,21050.50
max,199.00,480.00,540.00,311616436.45,1.00,1.01,691170668.78,1.11,1.11,1.01,4564530.12,1.01,1412633.25,1.01,79.90,26454.00


In [39]:
df[df['stock_id'] == 156].describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,10560.00,11100.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00
mean,156.00,258.50,270.00,2401325.51,0.14,1.00,14865161.85,1.00,1.00,1.00,110554.43,1.00,108356.79,1.00,0.42,14244.50
std,0.00,128.17,158.75,6005136.31,0.93,0.00,71001683.39,0.04,0.01,0.00,188906.71,0.00,149318.10,0.00,14.09,7049.59
min,156.00,37.00,0.00,0.00,-1.00,0.98,171907.17,0.75,0.96,0.98,18.27,0.98,7.80,0.98,-116.33,2035.00
25%,156.00,147.75,130.00,303630.36,-1.00,1.00,3911855.70,0.99,1.00,1.00,27591.36,1.00,29681.83,1.00,-6.38,8139.75
50%,156.00,258.50,270.00,1052156.43,1.00,1.00,6247247.91,1.00,1.00,1.00,66096.01,1.00,66678.52,1.00,-0.12,14244.50
75%,156.00,369.25,410.00,2691973.82,1.00,1.00,10346077.88,1.01,1.00,1.00,128549.52,1.00,131429.52,1.00,6.59,20349.25
max,156.00,480.00,540.00,204645278.59,1.00,1.03,1482126378.12,3.68,1.05,1.03,5137006.00,1.03,4706058.00,1.03,171.83,26454.00


In [40]:
# UTIL FUNCTION FOR GETTING THE NUMBER OF MISSING DATE
def get_stock_data(df, date_id, stock_id):
    return len(df[(df['date_id'] == date_id) & (df['stock_id'] == stock_id)])

def check_missing_days(df, stock_id, whether_print=False):
    DATE_RANGE = 480
    EXPECTED_DAYS = 55
    output_file_name = f'stock_data_{stock_id}.csv'
    missing_days = []

    if not os.path.exists(output_file_name):
        df[df['stock_id'] == stock_id].to_csv(output_file_name)

    for i in range(DATE_RANGE + 1):
        stock_data = get_stock_data(df, i, stock_id)
        if stock_data != EXPECTED_DAYS:
            if whether_print:
                print(i, stock_data)
            missing_days.append((i, stock_data))

    return missing_days

In [41]:
# check missing days for each stock with missing data
stocks_with_missing_dates = [69, 73, 78, 79, 99, 102, 135, 150, 153, 156, 199]
total = 0
for stock_id in stocks_with_missing_dates:
    missing_days = check_missing_days(df, stock_id)
    total += len(missing_days)
    print(f'stock_id {stock_id} and missing date: {missing_days}\n')
print(total)

stock_id 69 and missing date: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0)]

stock_id 73 and missing date: [(320, 0)]

stock_id 78 and missing date: [(0, 0), (1, 0), (2, 0), (3, 0)]

stock_id 79 and missing date: [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0), (46, 0), (47, 0), (48, 0), (49, 0), (50, 0), (51, 0), (52, 0), (53, 0), (54, 0), (55, 0), (56, 0), (57, 

In [42]:
df[df['stock_id'] == 153].describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,22605.00,22605.00,22605.00,22605.00,22605.00,22605.00,22605.00,9939.00,10275.00,22605.00,22605.00,22605.00,22605.00,22605.00,22605.00,22605.00
mean,153.00,275.00,270.00,838219.71,0.18,1.00,5127586.45,1.00,1.00,1.00,38539.69,1.00,36066.70,1.00,-0.57,15152.00
std,0.00,118.65,158.75,3745658.41,0.78,0.00,16992439.49,0.02,0.01,0.00,49912.45,0.00,56364.48,0.00,14.08,6525.65
min,153.00,70.00,0.00,0.00,-1.00,0.98,28199.39,0.67,0.91,0.97,18.39,0.98,16.27,0.97,-239.88,3850.00
25%,153.00,172.00,130.00,0.00,0.00,1.00,806580.55,1.00,1.00,1.00,11712.51,1.00,10640.50,1.00,-7.33,9501.00
50%,153.00,275.00,270.00,146564.03,0.00,1.00,1574067.40,1.00,1.00,1.00,24410.64,1.00,21804.20,1.00,-0.45,15152.00
75%,153.00,378.00,410.00,592194.06,1.00,1.00,2912713.12,1.00,1.00,1.00,47152.02,1.00,42324.15,1.00,6.19,20803.00
max,153.00,480.00,540.00,90502129.34,1.00,1.02,240725624.45,1.17,1.10,1.02,994182.25,1.02,1746364.10,1.02,139.27,26454.00


In [43]:
df[df['stock_id'] == 69].describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,10514.00,11100.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00
mean,69.00,258.50,270.00,1190557.34,0.17,1.00,4318962.44,1.00,1.00,1.00,225061.10,1.00,229451.30,1.00,-0.08,14244.50
std,0.00,128.17,158.75,4231205.96,0.86,0.00,14973285.30,0.04,0.01,0.00,196054.72,0.00,201613.21,0.00,12.51,7049.59
min,69.00,37.00,0.00,0.00,-1.00,0.97,255469.58,0.44,0.90,0.97,45.78,0.97,17.68,0.97,-107.65,2035.00
25%,69.00,147.75,130.00,29547.65,-1.00,1.00,1190966.31,1.00,1.00,1.00,100271.51,1.00,96921.61,1.00,-6.17,8139.75
50%,69.00,258.50,270.00,401185.54,0.00,1.00,2142625.87,1.00,1.00,1.00,180360.73,1.00,178837.55,1.00,-0.22,14244.50
75%,69.00,369.25,410.00,1088944.72,1.00,1.00,3596301.60,1.01,1.00,1.00,295145.85,1.00,305630.33,1.00,5.80,20349.25
max,69.00,480.00,540.00,117946398.66,1.00,1.02,275997753.91,2.09,1.10,1.02,4221028.35,1.02,2163217.21,1.02,203.05,26454.00


In [44]:
df[df['stock_id'] == 69].describe()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id
count,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,10514.00,11100.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00,24420.00
mean,69.00,258.50,270.00,1190557.34,0.17,1.00,4318962.44,1.00,1.00,1.00,225061.10,1.00,229451.30,1.00,-0.08,14244.50
std,0.00,128.17,158.75,4231205.96,0.86,0.00,14973285.30,0.04,0.01,0.00,196054.72,0.00,201613.21,0.00,12.51,7049.59
min,69.00,37.00,0.00,0.00,-1.00,0.97,255469.58,0.44,0.90,0.97,45.78,0.97,17.68,0.97,-107.65,2035.00
25%,69.00,147.75,130.00,29547.65,-1.00,1.00,1190966.31,1.00,1.00,1.00,100271.51,1.00,96921.61,1.00,-6.17,8139.75
50%,69.00,258.50,270.00,401185.54,0.00,1.00,2142625.87,1.00,1.00,1.00,180360.73,1.00,178837.55,1.00,-0.22,14244.50
75%,69.00,369.25,410.00,1088944.72,1.00,1.00,3596301.60,1.01,1.00,1.00,295145.85,1.00,305630.33,1.00,5.80,20349.25
max,69.00,480.00,540.00,117946398.66,1.00,1.02,275997753.91,2.09,1.10,1.02,4221028.35,1.02,2163217.21,1.02,203.05,26454.00


In [45]:
# EXPECTED DAYS TEMPLATE
DATE_RANGE = 480
EXPECTED_DAYS = 55
INVESTIGATE_STOCK_ID = 69
output_file_name = f'stock_data_{INVESTIGATE_STOCK_ID}.csv'

if not os.path.exists(output_file_name):
    df[df['stock_id'] == INVESTIGATE_STOCK_ID].to_csv(output_file_name)

for i in range(DATE_RANGE + 1):
    if len(df[(df['date_id'] == i) & (df['stock_id'] == INVESTIGATE_STOCK_ID)]) != EXPECTED_DAYS:
        print(i, len(df[(df['date_id'] == i) & (df['stock_id'] == INVESTIGATE_STOCK_ID)]))

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0


In [46]:
df[df['stock_id'] == 69]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
390569,69,37,0,1530007.23,1,1.00,840140.43,NaN,NaN,1.00,81488.82,1.00,35394.21,1.00,-4.64,2035,37_0_69
390763,69,37,10,1434155.16,1,1.00,935992.49,NaN,NaN,1.00,72852.31,1.00,56779.90,1.00,1.97,2036,37_10_69
390957,69,37,20,1434155.16,1,1.00,935992.49,NaN,NaN,1.00,83154.31,1.00,8675.90,1.00,1.36,2037,37_20_69
391151,69,37,30,1434155.16,1,1.00,935992.49,NaN,NaN,1.00,86588.31,1.00,29360.62,1.00,7.68,2038,37_30_69
391345,69,37,40,1434155.16,1,1.00,935992.49,NaN,NaN,1.00,86588.31,1.00,26096.42,1.00,7.19,2039,37_40_69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237049,69,480,500,2193328.35,-1,1.00,4325253.39,0.99,1.00,1.00,287827.34,1.00,454120.80,1.00,1.59,26450,480_500_69
5237249,69,480,510,2193328.35,-1,1.00,4325253.39,0.99,1.00,1.00,458622.57,1.00,374564.40,1.00,-0.99,26451,480_510_69
5237449,69,480,520,2127787.19,-1,1.00,4390794.54,0.99,1.00,1.00,465200.33,1.00,423981.60,1.00,-0.17,26452,480_520_69
5237649,69,480,530,2127787.19,-1,1.00,4390794.54,0.99,1.00,1.00,473749.74,1.00,388365.60,1.00,1.48,26453,480_530_69


In [48]:
df['target'].describe()

count   5237892.00
mean         -0.05
std           9.45
min        -385.29
25%          -4.56
50%          -0.06
75%           4.41
max         446.07
Name: target, dtype: float64